In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, log_loss
from skopt import forest_minimize
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

In [2]:
df = pd.read_csv('../data/raw_data_labeled.csv')

df = df[df['y'].notnull()]

In [3]:
df.isnull().sum()

watch-title                       0
y                                 0
watch-view-count                 21
watch-time-text                   0
watch-extras-section              0
content_watch-info-tag-list       0
og:image                          0
og:image:width                    0
og:image:height                   0
og:description                    0
og:video:width                   18
og:video:height                  18
og:video:tag                    198
channel_link_0                    8
Unnamed: 14                    1976
dtype: int64

In [4]:
# Clean date attribute
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].dropna().astype(str)
clean_date[2] = clean_date[2].dropna().astype(str)

month_map = {"jan": "Jan",
             "fev": "Feb",
             "mar": "Mar",
             "abr": "Apr",
             "mai": "May",
             "jun": "Jun",
             "jul": "Jul",
             "ago": "Aug",
             "set": "Sep",
             "out": "Oct",
             "nov": "Nov",
             "dez": "Dec"}

clean_date[1] = clean_date[1].map(month_map)
clean_date = clean_date.dropna().apply(lambda x: " ".join(x), axis=1)
clean_date = pd.to_datetime(clean_date, format="%d %b %Y")

# Clean view number
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False)
views = views.str.replace(".", "").fillna(0).astype(int)

In [5]:
features = pd.DataFrame()
y = df['y'].copy()

In [6]:
features['time_since_pub'] = (pd.to_datetime("2020-03-29") -  # HARDCODED
                              clean_date) / np.timedelta64(1, 'D')

# Extracting n of view feature
features['views'] = views

# Extracting n of view/day feature
features['views_per_day'] = features['views'] / features['time_since_pub']

# Droping time_since_pub to prevent bias
features = features.drop(['time_since_pub'], axis=1)

# Dropping problematic features
y = y[features.index]
df = df.loc[features.index]

In [7]:
resolutions = []
for height, width in zip(df['og:video:height'], df['og:video:width']):
    try:
        height = float(height)
        width = float(width)
    except:
        resolutions.append(np.nan)
        continue
        
    resolutions.append(height*width)
    
features['resolution'] = resolutions

In [8]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2275
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   views          2097 non-null   int64  
 1   views_per_day  2097 non-null   float64
 2   resolution     1957 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 65.5 KB


In [9]:
# Around 75% train and 25% to validation
split_date = '2020-02-27'
mask_train = (clean_date < split_date) & (clean_date.notnull())
mask_val = (clean_date >= split_date) & (clean_date.notnull())

X_train, X_val = features[mask_train.values], features[mask_val.values]
y_train, y_val = y[mask_train.values], y[mask_val.values]
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1827, 3), (270, 3), (1827,), (270,))

In [10]:
# Filling NaNs
X_train['resolution'] = X_train['resolution'].fillna(X_train['resolution'].mean())
X_val['resolution'] = X_val['resolution'].fillna(X_train['resolution'].mean())

/home/luis/anaconda3/envs/youtube_recommender/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/luis/anaconda3/envs/youtube_recommender/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
X_val.isnull().sum()

views            0
views_per_day    0
resolution       0
dtype: int64

In [23]:
# Extracting features from title
train_titles = df[mask_train]['watch-title']
val_titles = df[mask_val]['watch-title']

title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
title_bow_train = title_vec.fit_transform(train_titles)
title_bow_val = title_vec.transform(val_titles)

# Concat the BoW into features df
X_train_title = hstack([X_train, title_bow_train])
X_val_title = hstack([X_val, title_bow_val])

# Random Forest

In [24]:
# Random Forest Model
rfc = RandomForestClassifier(n_estimators=200, random_state=42,
                             class_weight="balanced", n_jobs=8)
rfc.fit(X_train_title, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=8,
                       oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [25]:
# Predicting
rf_train_proba = rfc.predict_proba(X_train_title)
rf_train_preds = rfc.predict(X_train_title)
rf_val_proba = rfc.predict_proba(X_val_title)
rf_val_preds = rfc.predict(X_val_title)

In [26]:
# Getting the metrics
print('TRAIN METRICS:')
print('log_loss: ', log_loss(y_train, rf_train_proba))
print('avg_precision_score: ', average_precision_score(y_train, rf_train_proba[:, 1]))
print('roc_auc: ', roc_auc_score(y_train, rf_train_proba[:, 1]))

print('\nVALIDATION METRICS:')
print('log_loss: ', log_loss(y_val, rf_val_preds))
print('avg_precision_score: ', average_precision_score(y_val, rf_val_proba[:, 1]))
print('roc_auc: ', roc_auc_score(y_val, rf_val_proba[:, 1]))

TRAIN METRICS:
log_loss:  0.1537853590465786
avg_precision_score:  0.9999978657105049
roc_auc:  0.9999993604600594

VALIDATION METRICS:
log_loss:  6.651992450614706
avg_precision_score:  0.4893890775561317
roc_auc:  0.792641975308642


# LGBM

In [13]:
lgbm = LGBMClassifier(random_state=42, class_weight="balanced", n_jobs=7)
lgbm.fit(X_train_title, y_train)

NameError: name 'X_train_title' is not defined

In [14]:
# Predicting
lgbm_train_proba = lgbm.predict_proba(X_train_title)
lgbm_train_preds = lgbm.predict(X_train_title)
lgbm_val_proba = lgbm.predict_proba(X_val_title)
lgbm_val_preds = lgbm.predict(X_val_title)

NameError: name 'X_train_title' is not defined

In [15]:
# Getting the metrics
print('TRAIN METRICS:')
print('log_loss: ', log_loss(y_train, lgbm_train_preds))
print('avg_precision_score: ', average_precision_score(y_train, lgbm_train_proba[:, 1]))
print('roc_auc: ', roc_auc_score(y_train, lgbm_train_proba[:, 1]))

print('\nVALIDATION METRICS:')
print('log_loss: ', log_loss(y_val, lgbm_val_preds))
print('avg_precision_score: ', average_precision_score(y_val, lgbm_val_proba[:, 1]))
print('roc_auc: ', roc_auc_score(y_val, lgbm_val_proba[:, 1]))

TRAIN METRICS:


NameError: name 'lgbm_train_preds' is not defined

In [17]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(train_titles)
    title_bow_val = title_vec.transform(val_titles)
    
    X_train_title = hstack([X_train, title_bow_train])
    X_val_title = hstack([X_val, title_bow_val])

    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=8)
    mdl.fit(X_train_title, y_train)
    
    p = mdl.predict_proba(X_val_title)[:, 1]
    
    print(roc_auc_score(y_val, p))
    
    return -average_precision_score(y_val, p)


space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          (1,5), # min_df
          (1,5)] # ngram_range

res = forest_minimize(tune_lgbm, space, random_state=42, n_random_starts=50, n_calls=75, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.03918194347141743, 8, 7, 0.4735411152109116, 0.14497617002710275, 558, 5, 4]


/home/luis/anaconda3/envs/youtube_recommender/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.7677037037037038
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.5326
Function value obtained: -0.4647
Current minimum: -0.4647
Iteration No: 2 started. Evaluating function at random point.
[0.0019307837536547132, 3, 2, 0.7358988336534836, 0.9416250735649627, 485, 4, 5]


/home/luis/anaconda3/envs/youtube_recommender/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.7790617283950617
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.5521
Function value obtained: -0.4456
Current minimum: -0.4647
Iteration No: 3 started. Evaluating function at random point.
[0.017177621112338382, 10, 12, 0.07190930378934497, 0.5485359272454697, 610, 2, 4]


...

0.8260740740740741
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 0.6733
Function value obtained: -0.5512
Current minimum: -0.5702
Iteration No: 74 started. Searching for the next optimal point.
[0.007457630914876315, 7, 5, 0.3697970957904218, 0.052825519874089254, 277, 5, 4]


/home/luis/anaconda3/envs/youtube_recommender/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.8232098765432098
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 0.6018
Function value obtained: -0.5224
Current minimum: -0.5702
Iteration No: 75 started. Searching for the next optimal point.
[0.009244970318559191, 9, 7, 0.5675826015697814, 0.10903735203292259, 103, 5, 4]


/home/luis/anaconda3/envs/youtube_recommender/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.8239012345679012
Iteration No: 75 ended. Search finished for the next optimal point.
Time taken: 0.5936
Function value obtained: -0.5227
Current minimum: -0.5702


In [18]:
lr, max_depth, min_child_samples, subsample, colsample_bytree, n_estimators, min_df, ngram_range = res.x

In [19]:
ngram_range = (1, ngram_range)
title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
title_bow_train = title_vec.fit_transform(train_titles)
title_bow_val = title_vec.transform(val_titles)

X_train_title = hstack([X_train, title_bow_train])
X_val_title = hstack([X_val, title_bow_val])

lgbm = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                     min_child_samples=min_child_samples, subsample=subsample,
                     colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                     class_weight='balanced', n_jobs=8)
lgbm.fit(X_train_title, y_train)

LGBMClassifier(bagging_freq=1, boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=0.05650259156907794, importance_type='split',
               learning_rate=0.0050661720343894556, max_depth=9,
               min_child_samples=6, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=575, n_jobs=8, num_leaves=512, objective=None,
               random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.18631641341650396, subsample_for_bin=200000,
               subsample_freq=0)

In [20]:
lgbm_train_proba = lgbm.predict_proba(X_train_title)
lgbm_train_preds = lgbm.predict(X_train_title)
lgbm_val_proba = lgbm.predict_proba(X_val_title)
lgbm_val_preds = lgbm.predict(X_val_title)

/home/luis/anaconda3/envs/youtube_recommender/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [21]:
# Getting the metrics
print('TRAIN METRICS:')
print('log_loss: ', log_loss(y_train, lgbm_train_preds))
print('avg_precision_score: ', average_precision_score(y_train, lgbm_train_proba[:, 1]))
print('roc_auc: ', roc_auc_score(y_train, lgbm_train_proba[:, 1]))

print('\nVALIDATION METRICS:')
print('log_loss: ', log_loss(y_val, lgbm_val_preds))
print('avg_precision_score: ', average_precision_score(y_val, lgbm_val_proba[:, 1]))
print('roc_auc: ', roc_auc_score(y_val, lgbm_val_proba[:, 1]))

TRAIN METRICS:
log_loss:  8.450496407118516
avg_precision_score:  0.7549747845229962
roc_auc:  0.8349865440796509

VALIDATION METRICS:
log_loss:  6.907864853444793
avg_precision_score:  0.5701757155143178
roc_auc:  0.8253827160493827


In [27]:
pd.DataFrame({"RF": rf_val_proba[:, 1], "LGBM": lgbm_val_proba[:, 1]}).corr()

,RF,LGBM
RF,1.000000,0.762598
LGBM,0.762598,1.000000


In [28]:
rf_weights = np.linspace(0,1,20)

for rf_weight in rf_weights:
    p = rf_weight*rf_val_proba[:, 1] + (1-rf_weight)*lgbm_val_proba[:, 1]
    print(rf_weight)
    print(average_precision_score(y_val, p), roc_auc_score(y_val, p))

0.0
0.5701757155143178 0.8253827160493827
0.05263157894736842
0.5712651161736446 0.8258765432098766
0.10526315789473684
0.5720701158167869 0.8266666666666667
0.15789473684210525
0.5695877907703422 0.8245925925925925
0.21052631578947367
0.5674371127484588 0.8241975308641976
0.2631578947368421
0.563471606640561 0.821037037037037
0.3157894736842105
0.5583955646950767 0.8194567901234567
0.3684210526315789
0.555255890111646 0.8165925925925926
0.42105263157894735
0.544290751611937 0.8126419753086421
0.47368421052631576
0.5249673089923815 0.8099753086419753
0.5263157894736842
0.5253522580292298 0.8074074074074074
0.5789473684210527
0.5182671358773665 0.8044444444444444
0.631578947368421
0.5105012628605115 0.8011851851851852
0.6842105263157894
0.5069279113810763 0.8000987654320988
0.7368421052631579
0.4953668899026871 0.7982222222222222
0.7894736842105263
0.49312970293514014 0.7969382716049384
0.8421052631578947
0.4876962913585326 0.7958518518518519
0.894736842105263
0.48644027299376 0.7947654

In [ ]:
p = 0.26*rf_val_proba[:, 1] + 0.74*lgbm_val_proba[:, 1]
print(average_precision_score(y_val, p), roc_auc_score(y_val, p))

In [ ]:
import joblib as jb
jb.dump(lgbm, "../pkls/lgbm_20200324.pkl.z")
jb.dump(rfc, "../pkls/rf_20200324.pkl.z")
jb.dump(title_vec, "../pkls/titlebow_20200324.pkl.z")